In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import finbox
import pandas as pd

In [3]:
spy = finbox.data.equity.get_history('SPY', fromdate='2018-01-01',
                                     todate='2018-03-15', set_index=True)
lqd = finbox.data.equity.get_history('LQD', fromdate='2018-01-01',
                                     todate='2018-03-15', set_index=True)
weights = pd.DataFrame({'SPY': 1, 'LQD': 0}, index=spy.index)
weights.iloc[2:5, :] = [.5, .5]

In [9]:
strat = finbox.backtest.build_weights_rebalance_strategy(
    tickers=['SPY', 'LQD'],
    weights=weights,
    datasets=[spy, lqd],
    lazy_rebalance=True,
    is_debug=False,
    commission_settings=[{'commission': 0.01}]
)

rb = finbox.backtest.ReportBuilder
rb = rb(strat, live_start_date=None)

In [5]:
from finbox.backtest.pyfolio import show_perf_stats

In [31]:
show_perf_stats(rb.returns, factor_returns=[rb.benchmark_rets],
                positions=rb.positions, transactions=rb.transactions,
                jupyter=True, live_start_date='2018-02-01')